In [ ]:
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
from torch.optim import SGD
import torch.onnx
import onnx
import onnxruntime

In [ ]:
D_in, H, D_out = 10, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

In [ ]:
outputs = model(torch.randn(64, 10))
outputs.shape

torch.Size([64, 10])

In [ ]:
df = pd.read_csv('riceClassification.csv')
X = df.iloc[:, :10]
Y = df.iloc[:, -1]

X.shape, Y.shape

((17995, 10), (17995,))

In [ ]:
x_2 = np.array(X)
y_2 = np.array(Y)

scaler = StandardScaler().fit(x_2)
x_2_normalized = scaler.transform(x_2)

X_train, X_test, y_train, y_test = train_test_split(x_2_normalized, y_2, test_size=0.2, random_state=42)

y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)


In [ ]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
X_t = torch.from_numpy(X_train).float()
Y_t = torch.from_numpy(y_train).long()

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = SGD(model.parameters(), lr=0.3)
epochs = 1000
log_each = 100
checkpoint_each = 20
l = []

for e in range(1, epochs + 1):
    y_pred = model(X_t)
    loss = loss_fn(y_pred, Y_t)
    l.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % log_each == 0:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

    if e % checkpoint_each == 0:
        torch.save({
            'epoch': e,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
        }, f"checkpoint_epoch_{e}.pt")

Epoch 100/1000 Loss 0.95812
Epoch 200/1000 Loss 0.79786
Epoch 300/1000 Loss 0.69009
Epoch 400/1000 Loss 0.60781
Epoch 500/1000 Loss 0.54303
Epoch 600/1000 Loss 0.49200
Epoch 700/1000 Loss 0.45137
Epoch 800/1000 Loss 0.41830
Epoch 900/1000 Loss 0.39083
Epoch 1000/1000 Loss 0.36760


In [ ]:
def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9649902750764101

Cargar todo los checkpoints basados en los epochs


In [ ]:
def evaluate(model, x):
    model.eval()
    y_pred = model(x)
    _, predicted = torch.max(y_pred, 1)
    return predicted

accuracies = []
for epoch in range(checkpoint_each, epochs + 1, checkpoint_each):
    checkpoint = torch.load(f"checkpoint_epoch_{epoch}.pt")
    model.load_state_dict(checkpoint['model_state_dict'])

    y_pred = evaluate(model, torch.from_numpy(X_test).float())
    accuracy = accuracy_score(y_test, y_pred.cpu().numpy())
    accuracies.append(accuracy)
    print(f"Accuracy at epoch {epoch}: {accuracy:.4f}")

print(f"Average accuracy: {np.mean(accuracies):.4f}")

Accuracy at epoch 20: 0.6238
Accuracy at epoch 40: 0.6916
Accuracy at epoch 60: 0.7549
Accuracy at epoch 80: 0.7944
Accuracy at epoch 100: 0.8194
Accuracy at epoch 120: 0.8044
Accuracy at epoch 140: 0.7610
Accuracy at epoch 160: 0.8033
Accuracy at epoch 180: 0.8122
Accuracy at epoch 200: 0.8183
Accuracy at epoch 220: 0.8277
Accuracy at epoch 240: 0.8355
Accuracy at epoch 260: 0.8433
Accuracy at epoch 280: 0.8497
Accuracy at epoch 300: 0.8572
Accuracy at epoch 320: 0.8661
Accuracy at epoch 340: 0.8733
Accuracy at epoch 360: 0.8825
Accuracy at epoch 380: 0.8911
Accuracy at epoch 400: 0.8994
Accuracy at epoch 420: 0.9086
Accuracy at epoch 440: 0.9147
Accuracy at epoch 460: 0.9203
Accuracy at epoch 480: 0.9241
Accuracy at epoch 500: 0.9283
Accuracy at epoch 520: 0.9319
Accuracy at epoch 540: 0.9347
Accuracy at epoch 560: 0.9372
Accuracy at epoch 580: 0.9400
Accuracy at epoch 600: 0.9422
Accuracy at epoch 620: 0.9447
Accuracy at epoch 640: 0.9461
Accuracy at epoch 660: 0.9483
Accuracy at ep

Cargar un Epoch a la vez


In [ ]:
def evaluate(model, x):
    model.eval()
    y_pred = model(x)
    _, predicted = torch.max(y_pred, 1)
    return predicted

checkpoint_file = "checkpoint_epoch_1000.pt"

checkpoint = torch.load(checkpoint_file)
model.load_state_dict(checkpoint['model_state_dict'])

y_pred = evaluate(model, torch.from_numpy(X_test).float())
accuracy = accuracy_score(y_test, y_pred.cpu().numpy())
print(f"Accuracy using {checkpoint_file}: {accuracy:.4f}")

Accuracy using checkpoint_epoch_1000.pt: 0.9650


Torchscript

In [ ]:
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, 'scripted_model.pt')

In [ ]:
scripted_model = torch.jit.load("scripted_model.pt")

def evaluate_model(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        predicted = model(X_test)
        _, predicted_labels = torch.max(predicted, 1)
        accuracy = accuracy_score(y_test, predicted_labels.numpy())
        return accuracy

test_accuracy = evaluate_model(scripted_model, X_test_tensor, y_test)
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.9650


ONNX


In [ ]:
dummy_input = torch.randn(10, 10)
torch.onnx.export(model, dummy_input, "model.onnx")